# Run Multiple Tools in a Pipeline

In this notebook, we explain how to use Arachne pipeline functionality.
In Arachne, we leverge an open-source Python framework for constructing pipelines (i.e., [Kedro](https://kedro.readthedocs.io/en/stable/index.html#)).
First, we describe how to use this feature from Arachne CLI, and then the way to execute from Python interface.

## Prepare a Model

For this tutorial, we will be working with the TFLite Converter and TVM for compling a Tensorflow (Keras) model by TVM after converting it into a TFLite model.
First, we prepare a Tensorflow model representing ResNet-50 v2 like the previous tutorails.

In [1]:
import tensorflow as tf

model = tf.keras.applications.resnet_v2.ResNet50V2()
model.summary()
model.save("/tmp/resnet50-v2.h5")

Model: "resnet50v2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
pool1_pad (ZeroPadding2D)       (None, 114, 114, 64) 0           conv1_conv[0][0]                 
_________________________________________________________________________________________

/workspaces/arachne/.venv/lib/python3.6/site-packages/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


## Construct and Run a Pipeline by `arachne.driver.pipeline`

Now, you are ready to run a pipeline by using `arachne.driver.pipeline` which is a CLI for constructing and executing a pipeline including multiple tools.
To define a pipeline, you have to specify the `pipeline` option that takes a list of tool names.
To configure the tool behavior, you can use tool specific options as well as `arachne.driver.cli`.

In [4]:
%%bash

python -m arachne.driver.pipeline input=/tmp/resnet50-v2.h5 output=/tmp/output.tar pipeline=[tflite_converter,tvm] tools.tflite_converter.ptq.method=fp16 tools.tvm.cpu_attr=[+fma,+avx2] tools.tvm.composite_target=[tensorrt,cpu]

[2022-03-21 14:36:40,019][tensorflow][WARNING] - No training configuration found in the save file, so the model was *not* compiled. Compile it manually.
[2022-03-21 14:36:41,794][tensorflow][WARNING] - No training configuration found in the save file, so the model was *not* compiled. Compile it manually.
[2022-03-21 14:36:41,798][tensorflow][WARNING] - Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.
call_node: 
free_var %input_1: Tensor[(1, 224, 224, 3), float32];
%0 = (%input_1,);
%1 = call_lowered(@tvmgen_default_fused_nn_pad, %0, metadata={"relay_attrs"={__dict__={"Primitive"=1, "hash"="95b394356d414c2f"}}, "all_prim_fn_vars"=['tvmgen_default_fused_nn_pad']}) /* ty=Tensor[(1, 230, 230, 3), float32] */;
%2 = @tvmgen_default_tensorrt_main_0(%1) /* ty=Tensor[(1, 64, 112, 112), float32] */;
%3 = (%2,);
%4 = call_lowered(@tvmgen_default_fused_nn_pad_1, %3, metadata={"relay_attrs"={__dic

/workspaces/arachne/.venv/lib/python3.6/site-packages/tensorflow/python/autograph/impl/api.py:22: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  import imp
2022-03-21 14:36:33.382610: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE3 SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-03-21 14:36:38.776234: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 373 MB memory:  -> device: 0, name: NVIDIA Tesla V100-SXM2-32GB, pci bus id: 0000:06:00.0, compute capability: 7.0
2022-03-21 14:36:38.779584: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/de

Here, we specify the two tools in a pipline (`pipeline=[tflite_converter,tvm]`). 
To configure the behavior of each tool, we can control it by modifying `tools.tflite_converter` and `tools.tvm` options.
In this example, the TFLite Converter first converts the input model in FP16 mode and the TVM compile the converted model for the TensorRT target with allowing to execute the remaining graph on CPU.

## Construct and Run a Pipeline by Python Interface

If you want to use pipeline functionality with Python interfaces, please import the `arachne.driver.pipeline` module.
First, you should setup the `arachne.driver.pipeline.PipelineConfig` object which is a config class for pipeline.
To specify the tools in pipeline, you should pass a list of tool names to `PipelineConfig.pipeline`.
The `arachne.driver.pipeline.get_default_tool_configs` is used for retrieving the default configs for specified configs and saving the result to `PipelineConfig.tools`.
To modify the behavior of each tool, you can change the value under `PipelineConfig.tools`.
Last, `arachne.driver.pipeline.run` is used for executing the pipeline.

In [9]:
from arachne.data import Model
from arachne.utils.model_utils import get_model_spec, save_model
from arachne.driver.pipeline import PipelineConfig, get_default_tool_configs, run

# Prepare an input model
model_path = "/tmp/resnet50-v2.h5"
input = Model(path=model_path, spec=get_model_spec(model_path))

# Construct a pipeline
cfg = PipelineConfig()
cfg.pipeline = ['tflite_converter', 'tvm']
cfg.tools = get_default_tool_configs(cfg.pipeline)

# Setup tflite_converter config
cfg.tools['tflite_converter'].ptq.method = "fp16"

# Setup tvm config
cfg.tools['tvm'].cpu_target = "x86-64"
cfg.tools['tvm'].cpu_attr = ['+fma', '+avx2']
cfg.tools['tvm'].composite_target = ['tensorrt', 'cpu']

output = run(input, cfg)

save_model(model=output, output_path="/tmp/output.tar", tvm_cfg=cfg.tools['tvm'])

{'tflite_converter': TFLiteConverterConfig(enable_tf_ops=False, allow_custom_ops=True, ptq=TFLiteConverterPTQConfg(method='fp16', representative_dataset=None)), 'tvm': TVMConfig(cpu_target='x86-64', cpu_attr=[], cpu_name=None, cuda_target_device='cuda', composite_target=['cpu'], target=None, target_host=None, desired_layout=None, disabled_pass=None, opt_level=3, export_format='tar', cross_compiler=None, cross_compiler_options=None)}
INFO:tensorflow:Assets written to: /tmp/tmp7vjtw3mv/assets


/workspaces/arachne/.venv/lib/python3.6/site-packages/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)
